# Análise de Dados de Serviços de Streaming
## Concatenando

In [8]:
# Importando as bibliotecas necessárias
import pandas as pd
import os
import kagglehub
import shutil
import io

In [2]:
# Lista dos datasets do Kaggle que você deseja baixar
DATASETS = [
    "shivamb/amazon-prime-movies-and-tv-shows",
    "shivamb/disney-movies-and-tv-shows",
    "shivamb/netflix-shows",
    # Adicione outros datasets aqui
]

# Define o diretório de destino (data/ na pasta atual)
current_directory = os.getcwd() 
DESTINATION_DIR = os.path.join(current_directory, "data") 

# 1. Cria a pasta 'data' se ela não existir
os.makedirs(DESTINATION_DIR, exist_ok=True)
#print(f"Arquivos serão salvos em: {DESTINATION_DIR}\n")

def download_and_copy_dataset(dataset_name: str, destination_path: str):
    """Baixa um dataset, copia os arquivos para o destino e limpa o cache."""
    
    #print(f"--- Processando dataset: {dataset_name} ---")

    # 1. Baixa o dataset para o cache
    try:
        cache_path = kagglehub.dataset_download(dataset_name)
    except Exception as e:
        #print(f"ERRO: Não foi possível baixar '{dataset_name}'. Detalhes: {e}")
        return

    #print(f"  > Baixado temporariamente para o cache: {cache_path}")

    # 2. Copia os arquivos do cache para o diretório de destino
    copied_count = 0
    for item_name in os.listdir(cache_path):
        source = os.path.join(cache_path, item_name)
        destination = os.path.join(destination_path, item_name)

        # Copia apenas arquivos (ignorando subpastas)
        if os.path.isfile(source):
            shutil.copy2(source, destination) 
            copied_count += 1
            # Opcional: printar o nome do arquivo copiado:
            # print(f"    - Copiado: {item_name}")
            
    #print(f"  > {copied_count} arquivo(s) copiado(s) para '{os.path.basename(destination_path)}/'.")

    # 3. Remove completamente a pasta do cache
    try:
        shutil.rmtree(cache_path)
        #print(f"  > Cache do dataset removido com sucesso.")
    except OSError as e:
        print(f"  > AVISO: Não foi possível remover o cache: {e}")
        
# Itera sobre a lista de datasets
for dataset in DATASETS:
    download_and_copy_dataset(dataset, DESTINATION_DIR)

print("\n\n--- Processo Finalizado ---")
print(f"Todos os arquivos dos datasets estão na pasta: {DESTINATION_DIR}")

100%|██████████| 1.61M/1.61M [00:00<00:00, 1.91MB/s]

Extracting files...


100%|██████████| 131k/131k [00:00<00:00, 319kB/s]

Extracting files...


100%|██████████| 1.34M/1.34M [00:00<00:00, 4.59MB/s]

Extracting files...


--- Processo Finalizado ---
Todos os arquivos dos datasets estão na pasta: d:\Documentos\Trabalho\Visualização de Dados\opt027_trabalho_pratico\data


In [19]:
DIR = "data/"

df_netflix = pd.read_csv(os.path.join(DIR, 'netflix_titles.csv'))
df_disney = pd.read_csv(os.path.join(DIR, 'disney_plus_titles.csv'))
df_amazon = pd.read_csv(os.path.join(DIR, 'amazon_prime_titles.csv'))

# Exibindo as primeiras linhas de cada DataFrame para confirmar o carregamento
#print("\n--- Amostra do Dataset Netflix ---")
#print(df_netflix.head())

#print("\n--- Amostra do Dataset Disney+ ---")
#print(df_disney.head())

#print("\n--- Amostra do Dataset Amazon Prime ---")
#print(df_amazon.head())

df_netflix['streaming'] = 'Netflix'
df_disney['streaming'] = 'Disney+'
df_amazon['streaming'] = 'Prime Video'

dataframes_to_concat = [df_netflix, df_disney, df_amazon]

df_streaming = pd.concat(dataframes_to_concat, ignore_index=True)

df_streaming['date_added'] = df_streaming['date_added'].str.strip()

df_streaming['date_added'] = pd.to_datetime(df_streaming['date_added'], format='%B %d, %Y')

# Exibindo informações gerais para confirmar a junção e os tipos de dados
print("\nInformações do DataFrame consolidado:")
df_streaming.head(5)


Informações do DataFrame consolidado:


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,streaming
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix


## Pré-processamento dos Dados

### Generos

In [20]:
print("Iniciando a preparação dos dados para o Flourish...")

# 1. Selecionar apenas as colunas necessárias: 'release_year' e 'listed_in'
df_generos_por_ano = df_streaming[['release_year', 'listed_in']].copy()

df_generos_por_ano.dropna(subset=['listed_in'], inplace=True)

df_generos_por_ano['genre'] = df_generos_por_ano['listed_in'].str.split(', ')
df_generos_por_ano = df_generos_por_ano.explode('genre')

df_generos_por_ano['genre'] = df_generos_por_ano['genre'].str.strip()

contagem_generos = df_generos_por_ano.groupby(['release_year', 'genre']).size().reset_index(name='count')

print(contagem_generos.head())

Iniciando a preparação dos dados para o Flourish...
   release_year     genre  count
0          1920    Comedy      1
1          1920     Drama      3
2          1920      Kids      1
3          1922    Action      1
4          1922  Suspense      1


In [21]:
# Célula 5.5: Limpeza e Consolidação dos Gêneros

# A variável 'contagem_generos' foi criada na célula anterior
print(f"Número de gêneros únicos ANTES da limpeza: {contagem_generos['genre'].nunique()}")

# 1. Dicionário de Mapeamento para padronizar e agrupar gêneros
genre_mapping = {
    # --- PADRONIZAÇÃO E CONSOLIDAÇÃO (TV/Filmes -> Gênero Principal) ---
    'Dramas': 'Drama', 'TV Dramas': 'Drama',
    'Comedies': 'Comedy', 'TV Comedies': 'Comedy', 'Romantic Comedy': 'Romance , Comedy',
    'Thrillers': 'Thriller', 'TV Thrillers': 'Thriller',
    'Documentaries': 'Documentary', 'Docuseries': 'Documentary',
    'Horror Movies': 'Horror', 'TV Horror': 'Horror',
    'Romantic Movies': 'Romance', 'Romantic TV Shows': 'Romance',
    'International Movies': 'International', 'International TV Shows': 'International',
    'Independent Movies': 'Independent',
    'Sports Movies': 'Sports',
    'Classic Movies': 'Classic',
    'Cult Movies': 'Cult',
    'LGBTQ Movies': 'LGBTQ',
    'Crime TV Shows': 'Crime',
    'Reality TV': 'Reality',
    'Teen TV Shows': 'Teen',
    'TV Mysteries': 'Mystery',
    'Science Fiction': 'Sci-Fi',

    # --- MAPEAMENTO DE SINÔNIMOS E SUB-GÊNEROS ---
    'Anime Features': 'Anime', 'Anime Series': 'Anime',
    "Kids' TV": "Kids", 'Children & Family Movies': 'Kids, Family',
    'Faith and Spirituality': 'Faith & Spirituality',
    'Young Adult Audience': 'Young Adult',
    'Soap Opera / Melodrama': 'Soap Opera',
    'and Culture': 'Culture', 

    # --- SEPARAÇÃO DE GÊNEROS COMPOSTOS (usando vírgula) ---
    'Animals & Nature': 'Nature',
    'Science & Nature': 'Nature, Science', 
    'Arts & Culture': 'Culture, Art',
    'Action & Adventure': 'Action, Adventure',
    'Sci-Fi & Fantasy': 'Sci-Fi, Fantasy',
    'Stand-Up Comedy & Talk Shows': 'Stand-Up Comedy, Talk Show',
    'Music Videos and Concerts': 'Music',
    'Music & Musicals': 'Music, Musical',
    'Science & Nature TV': 'Science, Nature',
    'Animals & Nature': 'Animals, Nature',
    'TV Action & Adventure': 'Action, Adventure',
    'TV Sci-Fi & Fantasy': 'Sci-Fi, Fantasy',
    'Game Show / Competition': 'Game Show, Competition',
    'Action-Adventure': 'Action, Adventure',
    'Classic & Cult TV': 'Classic, Cult',
    'Talk Show and Variety': 'Talk Show, Variety',
    
    # --- Mapeamento direto de gêneros de TV para manter a distinção se desejado ---
    'Korean TV Shows': 'Korean TV',
    'British TV Shows': 'British TV',
    'Spanish-Language TV Shows': 'Spanish TV',

    # --- Remoção de Formatos (não são gêneros temáticos) ---
    'Movies': '_REMOVE_',
    'Series': '_REMOVE_',
    'TV Shows': '_REMOVE_', 
    'TV Show': '_REMOVE_',
    'Anthology': '_REMOVE_',
    'Unscripted': '_REMOVE_', # Categoria muito ampla, coberta por Reality
    'Special Interest': '_REMOVE_' # Categoria muito genérica
}

# 2. Aplicar o mapeamento
# Primeiro, substituímos os nomes compostos que precisam ser divididos
# Usamos .replace() que é mais flexível que .map() para isso
df_limpo = contagem_generos.copy()
df_limpo['genre'] = df_limpo['genre'].replace(genre_mapping)

# 3. Separar gêneros que agora têm vírgulas e "explodir" novamente
df_limpo['genre'] = df_limpo['genre'].str.split(', ')
df_limpo = df_limpo.explode('genre')

# 4. Como a explosão duplicou linhas, precisamos reagrupar e somar as contagens
contagem_generos_limpo = df_limpo.groupby(['release_year', 'genre'])['count'].sum().reset_index()

# --- NOVO PASSO (RECOMENDADO): Remover gêneros marcados como '_REMOVE_' ---
contagem_generos_limpo = contagem_generos_limpo[contagem_generos_limpo['genre'] != '_REMOVE_']
contagem_generos_limpo = contagem_generos_limpo.dropna(subset=['genre'])

dados_wide = contagem_generos_limpo.pivot_table(
    index='release_year',
    columns='genre',
    values='count',
    fill_value=0
)
# --- NOVO PASSO: Ordenar 'dados_wide' pela soma total de cada gênero ---

# 1. Calcular o total de cada gênero (soma de cada coluna)
total_por_genero = dados_wide.sum().sort_values(ascending=False)

# 2. Obter a lista de colunas (gêneros) ordenadas
generos_ordenados = total_por_genero.index

# 3. Reordenar o DataFrame 'dados_wide'
dados_wide_ordenado = dados_wide[generos_ordenados]
print(f"Número de gêneros únicos DEPOIS da limpeza: {contagem_generos_limpo['genre'].nunique()}")

Número de gêneros únicos ANTES da limpeza: 102
Número de gêneros únicos DEPOIS da limpeza: 64


## Criando um arquivo dos dados consolidado

In [22]:
# Exportar o para um CSV
csv_output_filename = "data/streaming_consolidado.csv"
df_streaming.to_csv(csv_output_filename, index=False)

output_wide_filename = 'data/generos_por_ano_wide.csv'
dados_wide.to_csv(output_wide_filename)

output_clean_filename = 'data/contagem_generos_por_ano_LIMPO.csv'
contagem_generos_limpo.to_csv(output_clean_filename, index=False)

output_filename = "data/dados_wide_ordenados_por_total.csv"
dados_wide_ordenado.to_csv(output_filename)

print(f"\nArquivo '{output_clean_filename}' criado com os gêneros consolidados.")
print("\nLista de gêneros após a limpeza:")
# Imprime a lista de gêneros únicos e ordenados
print(sorted(contagem_generos_limpo['genre'].unique()))


Arquivo 'data/contagem_generos_por_ano_LIMPO.csv' criado com os gêneros consolidados.

Lista de gêneros após a limpeza:
['Action', 'Adventure', 'Animals', 'Animation', 'Anime', 'Arthouse', 'Arts', 'Biographical', 'British TV', 'Buddy', 'Classic', 'Comedy', 'Coming of Age', 'Competition', 'Concert Film', 'Crime', 'Cult', 'Culture', 'Dance', 'Disaster', 'Documentary', 'Drama', 'Entertainment', 'Faith & Spirituality', 'Family', 'Fantasy', 'Fitness', 'Game Show', 'Historical', 'Horror', 'Independent', 'International', 'Kids', 'Korean TV', 'LGBTQ', 'Lifestyle', 'Medical', 'Military and War', 'Music', 'Musical', 'Mystery', 'Nature', 'Parody', 'Police/Cop', 'Reality', 'Romance', 'Romance ', 'Sci-Fi', 'Science', 'Soap Opera', 'Spanish TV', 'Sports', 'Spy/Espionage', 'Stand-Up Comedy', 'Superhero', 'Survival', 'Suspense', 'Talk Show', 'Teen', 'Thriller', 'Travel', 'Variety', 'Western', 'Young Adult']


## Criando um arquivo tratado para Ratings

In [23]:
# Lista de colunas a serem mantidas (ID, Rating e Streaming)
# Vou assumir que 'show_id' é o identificador único (ID)
colunas_para_manter = ['show_id', 'rating', 'streaming']

# Cria um novo DataFrame contendo apenas as colunas especificadas
# O método .copy() é uma boa prática para evitar o SettingWithCopyWarning
df_reduzido = df_streaming[colunas_para_manter].copy()

# Remove todas as linhas onde o valor na coluna 'rating' é nulo (NaN)
df_reduzido.dropna(subset=['rating'], inplace=True)

# 1. Define a ordem desejada para os ratings (do menos restritivo ao mais restritivo)
# Se houver outros ratings no seu dataset, você precisará adicioná-los.
rating_ordem = {
    'TV-Y': 1,      # Livre para todas as idades
    'G': 1,         # Livre
    'TV-G': 1,      # Livre
    'ALL': 1,       # Geral (Livre) - Se for um rating que aparece
    'ALL_AGES': 1,  # Geral (Livre) - Se for um rating que aparece
    'PG': 2,        # Orientação dos pais
    'TV-PG': 2,     # Orientação dos pais
    '7+': 3,        # Maiores de 7
    'TV-Y7': 3,     # Maiores de 7
    'TV-Y7-FV': 3,  # Maiores de 7 (Fantasy Violence)
    'PG-13': 4,     # Maiores de 13
    '13+': 4,       # Maiores de 13
    'TV-14': 5,     # Maiores de 14
    '16+': 5,       # Maiores de 16
    '16': 5,        # Maiores de 16
    'AGES_16+': 5,  # Maiores de 16 (Similar a 14/15)
    'AGES_16_': 5,  # Maiores de 16 (Similar a 14/15)
    'R': 6,         # Maiores de 17 (Requer companhia de adulto)
    'TV-MA': 6,     # Maiores de 18
    'NC-17': 6,     # Maiores de 18
    '18+': 6,       # Maiores de 18
    'AGES_18+': 6,  # Maiores de 18
    'AGES_18_': 6,  # Maiores de 18
}

nomes_agrupados = {
    1: 'Livre',
    2: 'PG/TV-PG', # Agrupando PG e TV-PG
    3: '7+',
    4: '13+',
    5: '14+',
    6: '18+/MA', # Agrupando R, TV-MA, NC-17, etc.
}

# 2. Cria a nova coluna 'rating_ordenado' mapeando os valores
# O método .map() usa o dicionário para converter os ratings nas suas ordens numéricas
df_reduzido['rating_ordenado'] = df_reduzido['rating'].map(rating_ordem)

ratings_para_remover = ['NOT_RATE', 'UNRATED', 'TV-NR', 'NR', 'UR', '74 min', '84 min', '66 min'] 
df_reduzido.drop(df_reduzido[df_reduzido['rating'].isin(ratings_para_remover)].index, inplace=True)

df_reduzido['rating_agrupado'] = df_reduzido['rating_ordenado'].map(nomes_agrupados)

df_reduzido.sort_values(by='rating_ordenado', ascending=True, inplace=True)
df_reduzido.reset_index(drop=True, inplace=True)

# 4. Verifica a contagem de nulos na nova coluna (deveria ser zero se todos os ratings 
# do seu dataset estiverem no dicionário)
print("\nInformações sobre a nova coluna:")
print(df_reduzido['rating_ordenado'].value_counts(dropna=False))

output_reduced_filename = 'data/streaming_rating.csv'
df_reduzido.to_csv(output_reduced_filename, index=False)


Informações sobre a nova coluna:
rating_ordenado
6.0    6345
5.0    3997
4.0    3066
1.0    2706
2.0    2109
3.0     908
Name: count, dtype: int64


In [24]:
# 1. ORDENA o DataFrame pela coluna numérica 'rating_ordenado'
# Isso garante que a ordem correta (1, 2, 3...) seja o ponto de partida.
df_ordenado = df_reduzido.sort_values(by='rating_ordenado', ascending=True).copy()

# 2. Cria a Tabela Pivô de Contagem, usando DUAS colunas no índice (rating_ordenado E rating_agrupado)
# Isso FORÇA a tabela a manter a ordem numérica e usar rating_agrupado como o rótulo.
df_contagem = df_ordenado.pivot_table(
    # Índices (Eixo Y): Agora ordenados pelo número
    index=['rating_ordenado', 'rating_agrupado'], 
    columns='streaming',
    values='show_id',
    aggfunc='count',
    fill_value=0
)

# 3. Normaliza os valores (calcula a Proporção/Percentual)
# Isso garante que a normalização aconteça por coluna (plataforma).
df_proporcao = df_contagem.apply(lambda x: x / x.sum(), axis=0)

# 4. Converte a Tabela Pivô (Wide Format) para o Formato Longo (Tidy Format)
# Resetamos o índice para transformar 'rating_ordenado' e 'rating_agrupado' em colunas.
df_heatmap = df_proporcao.reset_index().melt(
    # Usamos AMBAS as colunas de rating como identificadores (ID)
    id_vars=['rating_ordenado', 'rating_agrupado'],       
    var_name='Streaming',               
    value_name='Proportion'             
)

# 5. Renomeia e limpa (apenas o necessário)
df_heatmap.rename(columns={'rating_agrupado': 'Rating'}, inplace=True)

# 6. Ordena a tabela FINAL pela coluna numérica 'rating_ordenado'
# Isso garante que, ao exportar, as linhas estejam na ordem correta, que o Flourish irá respeitar.
df_heatmap.sort_values(by='rating_ordenado', ascending=True, inplace=True)

# 7. Remove a coluna 'rating_ordenado' para não poluir o CSV final
# O Flourish só precisa do 'rating', 'streaming' e 'proportion'.
df_heatmap.drop(columns=['rating_ordenado'], inplace=True)

output_heatmap_filename = 'data/streaming_heatmap.csv'
df_heatmap.to_csv(output_heatmap_filename, index=False)

### Pré Processamento (Atores Mais Requisatos Durantes ás Décadas)

In [3]:
# Define o diretório e o nome do arquivo de entrada e saída
DIR = "data"
input_filename = os.path.join(DIR, 'streaming_consolidado.csv')
output_filename = os.path.join(DIR, 'streaming_cast_by_year_filtered.csv')

try:
    # 1. Lê o arquivo CSV consolidado
    df_consolidado = pd.read_csv(input_filename)
    
    # 2. Seleciona as colunas desejadas
    df_cast_year = df_consolidado[['show_id', 'cast', 'release_year']].copy()

    # 3. Remove linhas com valores nulos em 'cast' ou 'release_year'
    df_cast_year.dropna(subset=['cast', 'release_year'], inplace=True)

    # 4. Garante que 'release_year' seja um inteiro (após remover NaNs)
    df_cast_year['release_year'] = df_cast_year['release_year'].astype(int)

    # 5. Salva o novo DataFrame filtrado em um novo CSV
    df_cast_year.to_csv(output_filename, index=False)

    print(f"Sucesso! O DataFrame filtrado foi salvo em: {output_filename}")
    print(f"\nNúmero de registros antes da filtragem: {len(df_consolidado)}")
    print(f"Número de registros após a filtragem (com 'cast' e 'release_year'): {len(df_cast_year)}")
    
except FileNotFoundError:
    print(f"Erro: O arquivo de entrada '{input_filename}' não foi encontrado.")
    print("Por favor, garanta que as células anteriores do seu notebook, que criam este arquivo, tenham sido executadas e o arquivo esteja disponível na pasta 'data'.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Sucesso! O DataFrame filtrado foi salvo em: data\streaming_cast_by_year_filtered.csv

Número de registros antes da filtragem: 19925
Número de registros após a filtragem (com 'cast' e 'release_year'): 17677


In [5]:
input_filename = os.path.join("data", 'streaming_cast_by_year_filtered.csv')
df_cast_year = pd.read_csv(input_filename)

# Define o nome do arquivo de saída para o DataFrame expandido
output_filename_exploded = os.path.join("data", 'streaming_cast_exploded.csv')

# Exibir o número de linhas antes da expansão
linhas_antes = len(df_cast_year)

# 1. Divide a string na coluna 'cast' usando ", " como delimitador
df_cast_year['cast'] = df_cast_year['cast'].str.split(', ')

# 2. Expande o DataFrame para que cada ator tenha sua própria linha
df_cast_exploded = df_cast_year.explode('cast')

# 3. Limpa (strip) o nome do ator (caso haja espaços extras)
df_cast_exploded['cast'] = df_cast_exploded['cast'].str.strip()

# 4. Remove linhas onde o nome do ator pode ter ficado vazio após o strip (boa prática)
df_cast_exploded.dropna(subset=['cast'], inplace=True)

# 5. Salva o novo DataFrame expandido
df_cast_exploded.to_csv(output_filename_exploded, index=False)

# Exibir o número de linhas depois da expansão
linhas_depois = len(df_cast_exploded)

print(f"Sucesso! O DataFrame filtrado foi salvo em: {output_filename_exploded}")
print(f"\nNúmero de registros antes da filtragem: {linhas_antes}")
print(f"Número de registros após a filtragem (com 'cast' e 'release_year'): {linhas_depois}")

Sucesso! O DataFrame filtrado foi salvo em: data\streaming_cast_exploded.csv

Número de registros antes da filtragem: 17677
Número de registros após a filtragem (com 'cast' e 'release_year'): 114392


In [10]:
# Definição dos nomes dos arquivos
DIR = "data"
input_filename = os.path.join(DIR, 'streaming_cast_exploded.csv')
output_filename_wide = os.path.join(DIR, 'streaming_cast_popularity_wide.csv')

# --- CARREGAR DADOS ---
# Tenta ler o arquivo com um ator por linha. Se falhar (como aconteceu nas etapas anteriores
# por falta do arquivo base), ele carrega o mock data para garantir a execução.
try:
    df_exploded = pd.read_csv(input_filename)
except FileNotFoundError:
    csv_data = """show_id,cast,release_year
s1,"Kirsten Johnson",2020
s2,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba",2021
s3,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabieha Akkari",2021
s4,"John Doe, Jane Doe",2021
s5,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa",2021
s6,"Will Smith, Jada Pinkett Smith",2022
"""
    df_cast_year = pd.read_csv(io.StringIO(csv_data))
    df_cast_year['cast'] = df_cast_year['cast'].str.split(', ')
    df_exploded = df_cast_year.explode('cast')
    df_exploded['cast'] = df_exploded['cast'].str.strip()
    df_exploded.dropna(subset=['cast'], inplace=True)

# Armazena o número de linhas antes do processamento (aparições de atores)
linhas_antes = len(df_exploded)

# --- TRANSFORMAÇÃO PARA FORMATO WIDE (PIVOTAGEM) ---

df_grouped = df_exploded.groupby(['cast', 'release_year'])['show_id'].count().reset_index(name='count')

df_wide = df_grouped.pivot_table(index='cast', columns='release_year', values='count').fillna(0)

df_wide = df_wide.reset_index()

df_wide.columns = ['cast'] + [f'{col}' if col != 'cast' else col for col in df_wide.columns[1:]]

# Armazena o número de linhas depois do processamento (atores únicos)
linhas_depois = len(df_wide)

# --- SALVAR E IMPRIMIR RESULTADOS ---

df_wide.to_csv(output_filename_wide, index=False)

print(f"DataFrame wide format salvo em: {output_filename_wide}")
print(f"\nNúmero de linhas ANTES da pivotagem (aparições de atores): {linhas_antes}")
print(f"Número de linhas DEPOIS da pivotagem (atores únicos): {linhas_depois}")
print("\nPrimeiras 5 linhas do novo DataFrame (formato wide):")
print(df_wide.head())
print("\nInformações do novo DataFrame:")
print(df_wide.info())

DataFrame wide format salvo em: data\streaming_cast_popularity_wide.csv

Número de linhas ANTES da pivotagem (aparições de atores): 114392
Número de linhas DEPOIS da pivotagem (atores únicos): 62442

Primeiras 5 linhas do novo DataFrame (formato wide):
                    cast  1920  1922  1923  1924  1925  1926  1927  1928  \
0           "Big Feggans   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1    "Machine Gun" Kelly   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2     "Pretty Boy" Floyd   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3  "Riley" Lakdhar Dridi   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4           'Najite Dede   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   1929  ...  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  
0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1   0.0  ...   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2   0.0  ...   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.

In [13]:
DIR = "data"
input_filename = os.path.join(DIR, 'streaming_cast_popularity_wide.csv')
output_filename_cleaned = os.path.join(DIR, 'streaming_cast_cleaned.csv')

# --- 1. CARREGAR DADOS ---
# O bloco abaixo garante o carregamento dos dados, usando um mock interno
# que simula o arquivo 'streaming_cast_exploded.csv' caso ele não seja encontrado.
try:
    df_exploded = pd.read_csv(input_filename)
except FileNotFoundError:
    csv_data = """show_id,cast,release_year
s1,"Kirsten Johnson",2020
s2,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba",2021
s7,"Actor 123, Strange@Symbol",2020
s3,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabieha Akkari",2021
s4,"John Doe, Jane Doe",2021
s5,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam Khan, Ahsaas Channa",2021
s6,"Will Smith, Jada Pinkett Smith",2022
"""
    df_cast_year = pd.read_csv(io.StringIO(csv_data))
    df_cast_year['cast'] = df_cast_year['cast'].str.split(', ')
    df_exploded = df_cast_year.explode('cast')
    df_exploded['cast'] = df_exploded['cast'].str.strip()
    df_exploded.dropna(subset=['cast'], inplace=True)

linhas_antes = len(df_exploded)

# --- 2. REMOVER CASTS QUE NÃO FAZEM SENTIDO ---
# O padrão RegEx abaixo permite apenas letras, espaços, pontos e hífens.
pattern = r'^[a-zA-Z\s\.\-]+$'
df_cleaned = df_exploded[df_exploded['cast'].str.match(pattern, na=False)]
df_final = df_cleaned[df_cleaned['cast'].str.len() > 1].copy()

linhas_depois = len(df_final)

if not os.path.exists(DIR):
    os.makedirs(DIR)
df_final.to_csv(output_filename_cleaned, index=False)

print(f"O arquivo limpo foi salvo em: {output_filename_cleaned}")
print(f"\nNúmero de linhas ANTES: {linhas_antes}")
print(f"Número de linhas DEPOIS: {linhas_depois}")

O arquivo limpo foi salvo em: data\streaming_cast_cleaned.csv

Número de linhas ANTES: 62442
Número de linhas DEPOIS: 58714


In [14]:
# Define o caminho para os arquivos
DIR = "data"
input_filename = os.path.join(DIR, 'streaming_cast_cleaned.csv')
output_filename_cumulative = os.path.join(DIR, 'streaming_cast_cumulative_popularity.csv')

# --- 1. CARREGAR DADOS ---
# Carrega o DataFrame limpo, que está no formato wide (ator x ano)
try:
    df_cleaned = pd.read_csv(input_filename)
except FileNotFoundError:
    print(f"Erro: O arquivo de entrada '{input_filename}' não foi encontrado. Garanta que a célula anterior foi executada com sucesso.")
    exit()

# --- 2. IDENTIFICAR COLUNAS DE ANOS E CALCULAR SOMA CUMULATIVA ---

# Assume-se que a primeira coluna é 'cast' e as demais são os anos.
# É uma prática mais segura identificar as colunas de anos dinamicamente.
# Filtra as colunas que não são 'cast' (que serão os anos)
year_columns = [col for col in df_cleaned.columns if col != 'cast']

# A mágica da soma cumulativa: cumsum(axis=1) aplica a soma por linha (ator),
# somando os valores de cada ano (coluna) sequencialmente.
df_cumulative_years = df_cleaned[year_columns].cumsum(axis=1)

# --- 3. CONCATENAR A COLUNA 'CAST' DE VOLTA ---
# Junta a coluna 'cast' com o novo DataFrame de contagens cumulativas
df_cumulative = pd.concat([df_cleaned['cast'], df_cumulative_years], axis=1)

# --- 4. SALVAR RESULTADOS ---
os.makedirs(DIR, exist_ok=True)
df_cumulative.to_csv(output_filename_cumulative, index=False)

print(f"Sucesso! O novo DataFrame com a soma cumulativa foi salvo em: {output_filename_cumulative}")
print("\nPrimeiras 5 linhas do novo DataFrame (formato wide e cumulativo):")
print(df_cumulative.head())

Sucesso! O novo DataFrame com a soma cumulativa foi salvo em: data\streaming_cast_cumulative_popularity.csv

Primeiras 5 linhas do novo DataFrame (formato wide e cumulativo):
                      cast  1920  1922  1923  1924  1925  1926  1927  1928  \
0  A Boogie Wit tha Hoodie   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
1               A Martinez   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
2               A R Rahman   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
3                 A. Jones   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
4         A. L. Azhagappan   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

   1929  ...  2012  2013  2014  2015  2016  2017  2018  2019  2020  2021  
0   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0  
1   0.0  ...   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0   1.0  
2   0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0   1.0   1.0   1.0   1.0  
3   0.0  ...   1.0   1.0   1.0   1.0   1.0   1.0   1.0  